In [7]:
from keras_image_helper import create_preprocessor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input
import numpy as np

In [ ]:
# https://github.com/alexeygrigorev/mlbookcamp-code/releases/download/chapter7-model/xception_v4_large_08_0.894.h5 -O clothing-model.h5
# !wget https://raw.githubusercontent.com/alexeygrigorev/clothing-dataset-small/master/test/pants/4aabd82c-82e1-4181-a84d-d0c6e550d26d.jpg -O pants.jpg

In [35]:
classes = [
    'dress'
    , 'hat'
    , 'longsleeve'
    , 'outwear'
    , 'pants'
    , 'shirt'
    , 'shoes'
    , 'shorts'
    , 'skirt'
    , 't-shirt'
]

### Original Tensorflow Model

In [16]:
model = keras.models.load_model('clothing-model.h5')

In [14]:
img = load_img('pants.jpg', target_size=(299,299))
x = np.array(img)
X = np.array([x])
X = preprocess_input(X)

In [17]:
preds = model.predict(X)
dict(zip(classes, preds[0]))

1/1 [==============================] - 0s 481ms/step


### Converted Tensorflow Model

In [25]:
import tensorflow.lite as tflite
# import tflite_runtime as tflite - without installing tensorflow


In [26]:
interpreter = tflite.Interpreter(model_path = 'clothing-model-v4.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']


In [36]:
# X is still using keras preprocessing / img load
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
dict(zip(classes, preds[0]))

{'dress': -1.8682901,
 'hat': -4.7612457,
 'longsleeve': -2.316981,
 'outwear': -1.062568,
 'pants': 9.887158,
 'shirt': -2.8124313,
 'shoes': -3.666284,
 'shorts': 3.2003634,
 'skirt': -2.6023405,
 't-shirt': -4.8350444}

### Removing TF Dependencies

In [37]:
from PIL import Image

In [47]:
with Image.open('pants.jpg') as img:
    img = img.resize((299,299), Image.NEAREST)

In [39]:
### replacing xception preprocess input by finding the function in library:
### https://github.com/keras-team/keras/blob/master/keras/applications/imagenet_utils.py

def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [42]:
x = np.array(img, dtype='float32')
X = np.array([x])
X = preprocess_input(X)

In [43]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
dict(zip(classes, preds[0]))

{'dress': -1.8682901,
 'hat': -4.7612457,
 'longsleeve': -2.316981,
 'outwear': -1.062568,
 'pants': 9.887158,
 'shirt': -2.8124313,
 'shoes': -3.666284,
 'shorts': 3.2003634,
 'skirt': -2.6023405,
 't-shirt': -4.8350444}